<a href="https://colab.research.google.com/github/avery02022/CS/blob/main/Karatsuba_Multiplication_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
time: 513 µs (started: 2024-08-06 03:34:29 +00:00)


## Bitwise representation of numbers

We will represent numbers in binary as a sequence of 1s and 0s.
Recall  the basics of converting decimal numbers into binary and vice versa.

### Example
The number 5 in decimal is written as $101$ in binary:
$5_{10} = 101_2 $


| Decimal | Binary |
|---|---|
| 5 | 101 |
| 12 | 1100 |
| 16 | 10000 |
| 23 | 10111 |

We will represent binary numbers as a list but the least significant bit is the first element and most significant bit is the last element.
Eg., the number $(1011)_2$ is written in a list as `[1, 1, 0, 1]`.

In [2]:
def convert_to_binary(n):
    assert n >= 0
    if n == 0:
        return [0]
    lst = []
    while n > 0:
        lst.append( n % 2)
        n = n // 2 # Integer division in python uses //
    return lst

def convert_to_decimal(lst):
    sum = 0
    f = 1
    for elt in lst:
        sum = sum + elt * f
        f = f * 2
    return sum

time: 981 µs (started: 2024-08-06 03:38:26 +00:00)


In [3]:
print(f'6 = {convert_to_binary(6)}')
print(f'23 = {convert_to_binary(23)}')
print(f'46 = {convert_to_binary(46)}')
print(f'128 = {convert_to_binary(128)}')
print(f'71 = {convert_to_binary(71)}')
print(convert_to_decimal([1, 0, 1, 1, 0, 1])) # should be 45
print(convert_to_decimal([0, 1, 1, 0, 1])) # should be 22

6 = [0, 1, 1]
23 = [1, 1, 1, 0, 1]
46 = [0, 1, 1, 1, 0, 1]
128 = [0, 0, 0, 0, 0, 0, 0, 1]
71 = [1, 1, 1, 0, 0, 0, 1]
45
22
time: 6.1 ms (started: 2024-08-06 03:38:31 +00:00)


## Addition and Subtraction

We will implement addition and subtraction for binary numbers.
Addition was discussed in the lecture. For subtraction, we use a very nice trick called twos-complement method that turns subtraction into addition after flipping 1s and 0s.
See here for a quick explanation:

https://www.geeksforgeeks.org/subtraction-of-two-numbers-using-2s-complement/

In [10]:
def bitwise_add(ai, bi, ci):
    if ai == 0:
        if bi == 0:
            return (ci, 0)
        else: # ai= 0, bi = 1
            return (1-ci, ci)
    else:
        if bi == 0:
            return (1-ci, ci)
        else:
            return (ci, 1)

def add(a, b):
    # add bit strings a, b
    (n, m) = len(a), len(b)
    carry = 0
    c = []
    for i in range(max(m,n)):
        ai = a[i] if i < n else 0
        bi = b[i] if i < m else 0
        (ci, carry) = bitwise_add(ai, bi, carry)
        c.append(ci)
    if carry == 1:
        c.append(carry)
    return c

def subtract(a, b):
    # we will use two's complement subtraction
    # this is a very nice and common trick where
    # we can use addition to perform subraction of
    # binary numbers. It is used inside computers.
    # assume a >= b -- this will generally hold for all our use cases
    n = len(a)
    #assert(len(b) <= n)
    k = len(a) - len(b)
    bcomp = [1-elt for elt in b] + [1]*k # flip the bits in b and pad with 1s
    bcomp2 = add(bcomp, [1]) # add 1
    r = add(a, bcomp2)
    return r[0:n]

def pad(a, k):
    return  [0]*k + a

In [28]:
print(subtract([1,1,1],[1])) # should be [0, 1, 1]
print([1-elt for elt in [1]] + [1]*2)

[0, 1, 1]
[0, 1, 1]


In [17]:
print(add([1,0,1,1,0], [1, 0, 0, 0, 1])) # should be 0, 1, 1, 1, 1
print(add([0, 1, 1], [1, 1, 1])) # should be 1, 0, 1, 1
print(add([0, 1], [1, 1, 1])) # should be 1, 0, 0, 1
print(add([0], [1,0,1,0,1,1,0,1])) # should be 1, 0, 1, 0, 1, 1, 0, 1

[0, 1, 1, 1, 1]
[1, 0, 1, 1]
[1, 0, 0, 1]
[1, 0, 1, 0, 1, 1, 0, 1]


In [18]:
print(subtract([1,1,1],[1])) # should be [0, 1, 1]
print(subtract([1,0,1], [0, 1])) # should be [1, 1, 0]
print(subtract([0, 0, 0, 1], [1, 1])) # should be [1, 0, 1, 0]
print(subtract([0, 1, 0, 1], [1, 0, 0, 1])) # should be [1, 0, 0, 0]
print(subtract([0, 1, 0, 1, 1, 1, 0, 1],[0]))

[0, 1, 1]
[1, 1, 0]
[1, 0, 1, 0]
[1, 0, 0, 0]
[0, 1, 0, 1, 1, 1, 0, 1]


In [30]:
def grade_school_multiply(a, b):
    n, m = len(a), len(b)
    tmp = a
    res = [0]
    for bit in b:
        if bit == 1:
            res = add(res, tmp)
        tmp = [0]+tmp # shift tmp
    return res

In [31]:
print(grade_school_multiply([1, 0, 1], [0, 1])) #  should be 0, 1, 0, 1
print(grade_school_multiply([0, 1, 1], [1, 1])) # should be 0, 1, 0, 0 , 1
print(grade_school_multiply([0, 0, 1], [1, 0, 1])) # should be 0, 0, 1, 0 , 1
print(grade_school_multiply([0, 0, 0, 1], [1, 0, 1])) # should be 0, 0, 0, 1, 0, 1

[0, 1, 0, 1]
[0, 1, 0, 0, 1]
[0, 0, 1, 0, 1]
[0, 0, 0, 1, 0, 1]


In [38]:
def karatsuba_multiply(a, b):
    (m, n) = len(a), len(b)
    if m <= 2 or n <= 2:
        # revert to grade school multiplication
        return grade_school_multiply(a, b)
    else:
        mid1 = m//2
        a1 = a[0:mid1]
        a2 = a[mid1:]
        b1 = b[0:mid1]
        b2 = b[mid1:]
        # [a] = 2^{mid1} * [a2] + [a1]
        # [b] = 2^{mid1} * [b2] + [b1]
        # [a]* [b] = 2^{2*mid1} ([a2]*[b2]) + 2^mid1 ([a2]*[b1] + [a2]*[b1]) + [a1]*[b1]

        # 3 recursive calls to karatsuba_multiply
        r1 = karatsuba_multiply(a1, b1)
        r2 = karatsuba_multiply(a2, b2)
        r3 = karatsuba_multiply(add(a1, a2), add(b1, b2))
        # Do subtraction
        r4a = subtract(r3, r1)
        r4 = subtract(r4a, r2)

        # Do paddding
        s1 = pad(r2, 2*mid1)
        s2 = pad(r4, mid1)
        s3 = add(s1, s2)
        return add(s3, r1)

In [62]:
print(karatsuba_multiply([0, 0, 1, 1], [1, 1]))


[0, 0, 1, 0, 0, 1]


In [33]:
print(karatsuba_multiply([0, 0, 0, 1], [1, 0, 1])) # should be 0, 0, 0, 1, 0, 1
print(karatsuba_multiply([0, 0, 1], [1, 0, 1])) # should be 0, 0, 1, 0 , 1

[0, 0, 0, 1, 0, 1]
[0, 0, 1, 0, 1]


In [34]:
print(grade_school_multiply([1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0], [0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,1]))

[0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1]


In [35]:
print(karatsuba_multiply([1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0], [0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,1]))

[0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1]


In [36]:
from random import randint
a = [randint(0, 1) for j in range(10000)]
b = [randint(0, 1) for j in range(10000)]


In [37]:
c = grade_school_multiply(a,b)

In [ ]:
c = karatsuba_multiply(a, b)

time: 23.1 s (started: 2021-05-13 22:16:26 -06:00)
